In [ ]:
#농촌진흥청에서 1인당 연간소비량을 1주 기준으로 단순 변환하고, 계절별 요인이 적용한다 생각하여, 수요공급에 따라 가격이 비싼것과 소비량이 상관이 있을거라 판단해
# 농넷에서 관련 농산품 월별 가격변동 데이터를 가져와 계절로 치환하고 가장 높은 값을 기준으로 가중치를 부여해 결과를 출력했습니다. (인구를 변경하면 그에 따라 다른 값을 출력합니다)
# 추후에 데이터를 얻는다면 딥러닝을 이용하거나, 실제 방문했을때 총 수요량과 이데이터의 차이값과 주문받은(카드+앱)데이터를 비교한다면 점차 오차를 발견할 수 있을것같습니다.

In [80]:
import pandas as pd
import numpy as np

def calculate_and_apply_weights_with_provided_data(data_path, population_input, seed=None):
    data = pd.read_csv("/content/drive/MyDrive/주별채소과일소비량.csv", encoding='EUC-KR')
    columns_to_remove = ["Year_주별_구매량", "총채소_주별_구매량", "채소기타_주별_구매량", "총과일_주별_구매량", "과일기타_주별_구매량"]
    data = data.drop(columns=columns_to_remove, errors='ignore')

    if seed is not None:
        np.random.seed(seed)

    # 구매량 계산
    data_with_purchases = data.copy()
    for col in data.columns[1:]:
        probabilities = data[col]
        total_purchases = []

        for prob in probabilities:
            prob = np.clip(prob, 0, 1)  # 확률 값 제한
            purchases = np.random.binomial(1, prob, population_input).sum()
            total_purchases.append(purchases)

        data_with_purchases[col] = total_purchases

    # 2015-2022 평균값 계산
    averages = data_with_purchases.iloc[0:8].mean()

    # 가중치 데이터
    weights_data = {
        '품목': ['고추', '복숭아', '포도', '마늘', '배', '사과', '양파', '감귤', '단감', '배추', '무'],
        '겨울': [1.0, 1.0, 1.0, 1.0, 0.979592, 1.0, 1.0, 1.0, 1.0, 0.964492, 0.761671],
        '봄': [0.737057, 0.906250, 0.902050, 0.865922, 1.0, 0.926136, 0.983412, 0.919431, 0.875676, 0.946294, 0.857494],
        '여름': [0.407664, 0.666667, 0.633257, 0.698324, 0.897959, 0.801136, 0.824645, 0.824645, 0.778378, 1.0, 1.0],
        '가을': [0.559723, 0.757812, 0.712984, 0.743017, 0.906122, 0.698864, 0.736967, 0.763033, 0.681081, 0.806480, 0.977887],
    }

    # 평균값 데이터프레임 생성
    averages_df = pd.DataFrame(averages.items(), columns=['품목_주별_구매량', '평균값'])
    averages_df['품목'] = averages_df['품목_주별_구매량'].str.split('_').str[0]

    # 가중치 데이터프레임 생성
    weights_df = pd.DataFrame(weights_data)

    # 평균값에 가중치 적용
    merged_df = pd.merge(averages_df, weights_df, on='품목')
    for season in ['겨울', '봄', '여름', '가을']:
        merged_df[f'{season}_가중치_적용값'] = merged_df['평균값'] * merged_df[season]

    # 최종 결과 출력
    result_df = merged_df[['품목', '겨울_가중치_적용값', '봄_가중치_적용값', '여름_가중치_적용값', '가을_가중치_적용값']]
    print("계절별 가중치 적용 결과:")
    print(result_df)

# 데이터 파일 경로 지정
population_input = 1200  # 예시 인구수
calculate_and_apply_weights_with_provided_data(data_path, population_input, seed=42)

###  추후에 사용자 데이터가 나오면 더 정확하게 진행할 계획

계절별 가중치 적용 결과:
     품목   겨울_가중치_적용값    봄_가중치_적용값   여름_가중치_적용값  가을_가중치_적용값
0     무   399.020395   449.219669   523.875000  512.290552
1    배추  1102.414356  1081.614042  1143.000000  921.806640
2    마늘   165.375000   143.201851   115.485331  122.876436
3    양파   649.375000   638.603167   535.503847  478.567946
4    고추    70.625000    52.054651    28.791270   39.530437
5    사과   242.375000   224.472213   194.175338  169.387162
6     배    81.551034    83.250000    74.755087   75.434657
7   복숭아    84.250000    76.351562    56.166695   63.845661
8    포도   109.125000    98.436206    69.104170   77.804379
9    단감    42.750000    37.435149    33.275660   29.116213
10   감귤   283.750000   260.888546   233.993019  216.510614
